In [2]:
#!pip install tensorflow

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical


ModuleNotFoundError: No module named 'tensorflow.python'

In [4]:
# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images to the range of 0-1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape dataset to have a single channel
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


NameError: name 'mnist' is not defined

In [3]:
# Define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 corresponds to the number of classes (digits 0-9)


NameError: name 'Sequential' is not defined

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)


Epoch 1/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.1487 - accuracy: 0.9563 - val_loss: 0.0584 - val_accuracy: 0.9813
Epoch 2/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0524 - accuracy: 0.9840 - val_loss: 0.0484 - val_accuracy: 0.9847
Epoch 3/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0325 - accuracy: 0.9897 - val_loss: 0.0434 - val_accuracy: 0.9845
Epoch 4/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0223 - accuracy: 0.9931 - val_loss: 0.0451 - val_accuracy: 0.9847
Epoch 5/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0150 - accuracy: 0.9951 - val_loss: 0.0495 - val_accuracy: 0.9847
Epoch 6/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0111 - accuracy: 0.9963 - val_loss: 0.0438 - val_accuracy: 0.9866
Epoch 7/10
1875/1875 [==============================] - 38s 21ms/step - loss: 0.0079 - accuracy: 0.9974 - val_loss: 0.0526 -

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


313/313 [==============================] - 2s 5ms/step - loss: 0.0545 - accuracy: 0.9873
Test accuracy: 0.9872999787330627


In [10]:
model.save('mnist_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Web Service Creation

In [12]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from PIL import Image
import io


In [13]:
model = tf.keras.models.load_model('mnist_model.h5')


In [14]:
app = Flask(__name__)


In [15]:
@app.route('/predict', methods=['POST'])
def predict():
    # Ensure an image was provided
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']

    # Read the image
    image = Image.open(io.BytesIO(file.read()))
    image = image.resize((28, 28)).convert('L')  # Resize and convert to grayscale
    image = np.array(image)
    image = image.reshape(1, 28, 28, 1)  # Reshape for model input
    image = image / 255.0  # Normalize the image

    # Make a prediction
    prediction = model.predict(image)
    predicted_digit = np.argmax(prediction)

    return jsonify({'digit': int(predicted_digit)})


In [16]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
